In [166]:
"""
Simple class to detect collisions.
Can miss directory collisions in path. Ex.
    - BAR
    - bar/file
"""
class Collisions:    
    def __init__(self):
        self.records = {}
    
    def __normalize(self, path):
        return path.lower()

    def record(self, path):
        npath = self.__normalize(path)
        if npath in self.records.keys():
            self.records[npath] += [path]
        else:
            self.records[npath] = [path]
    
    def __report(self, arr):
        print(arr)
    
    def process(self):
        for k in self.records.keys():
            if len(self.records[k]) > 1:
                self.__report(self.records[k])

In [108]:
"""
Break path into components and find collisions.
This approach can spot directory collisions.

TODO: Print prefix of colliding paths
"""
class Tree:
    boolean_str = {
        True: "✓",
        False: "x"
    }

    def __init__(self, label="/"):
        self.label = label
        self.tainted = False # represents collision
        self.children = []
    
    def __repr__(self):
        return "{} {}".format(Tree.boolean_str[self.tainted], self.label)

    def __eq__(self, other):
        if isinstance(other, Tree):
            return self.label.lower() == other.label.lower()
        return False
    
    def helper_path_to_arr(self, path):
        arr = path.split("/")

        if len(arr) > 0 and arr[len(arr)-1] == "":
            arr.pop() # remove last entry of ''
        return arr
    
    def add_node(self, n):
        newnode = Tree(n)
        for c in self.children:
            if c == newnode:
                # Collision?
                if c.label != newnode.label:
                    #print("tainted:", newnode.label)
                    c.tainted = True
                return c
        
        # Not an existing child
        self.children += [newnode]
        return newnode
    
    def add_nodes(self, arr):
        parent = self
        for n in arr:
            parent = parent.add_node(n)
    
    def add_path(self, path):
        arr = self.helper_path_to_arr(path)
        return self.add_nodes(arr)
    
    """
    Print out all tainted paths
    """
    def walk(self, path="", tainted=False):
        # update path
        if self.label != "/":
            path = path + "/" + self.label
    
        # leaf node
        if len(self.children) == 0:
            print(Tree.boolean_str[tainted], path[1:].lower()) # remove "/" prefix
            return

        # intermediate nodes
        for c in self.children:
            t = tainted | c.tainted
            c.walk(path, t)
    
    """
    Does this path have collision?
    """
    def __has_collision(self, arr):
        if len(arr) == 0:
            return False
        
        # get first element
        t = Tree(arr[0])
        for c in self.children:
            if c == t and c.tainted: # found node?
                return True
            elif c == t and not c.tainted:
                return c.__has_collision(arr[1:])
        return False
    
    def has_collision(self, path):
        arr = self.helper_path_to_arr(path)
        return self.__has_collision(arr)


In [118]:
inputs = [
    "dir/foo",
    "dir/FOO",
    "DIR/foo",
    
    "path/BAR", # will be missed by Collisions()
    "path/bar/input",
    
    "path/dir/file1",
    "path/dir/file2",
    
    "point/",
    "Point",
    "POINT/",
    
    "clean/path1",
    "clean/path2",
]

def driver1():
    c = Collisions()
    for i in inputs:
        c.record(i)
    c.process()

def driver2():
    t = Tree()

    # Build tree
    for i in inputs:
        t.add_path(i)
    print("Tainted paths:")
    t.walk()
    print()

    # Test componets
    print("Colliding paths:")
    for i in inputs:
        if t.has_collision(i):
            print("-", i)

driver2()

Tainted paths:
✓ dir/foo
✓ path/bar/input
x path/dir/file1
x path/dir/file2
✓ point
x clean/path1
x clean/path2

Colliding paths:
- dir/foo
- dir/FOO
- DIR/foo
- path/BAR
- path/bar/input
- point/
- Point
- POINT/


In [133]:
import os

def analyze_tarball(filename):
    stream = os.popen("{} {}".format("tar tf", filename))
    paths = stream.readlines()
    paths = [p.strip("\n") for p in paths]
    print("Contents:", paths)
    
    # Analyze archive
    t = Tree()
    for p in paths:
        t.add_path(p)

    # Fetch collisions
    collisions = []
    for i in paths:
        if t.has_collision(i):
            collisions += [i]

    # Report collisions
    if len(collisions) > 1:
        print("Collisions:")
        for c in collisions:
            print("-", c)

analyze_tarball("tarball")

Contents: ['dir/', 'dir/file2', 'dir/file', 'dir/FILE']
Collisions:
- dir/file
- dir/FILE
